In [43]:
class Graph:
    """Graph implemented with adjacency lists.
    
    Assuming the total number of edges is known, we can order (with
    arbitrary order) the edges leaving from each vertex.
    
    We use three arrays:
      * The first one is a map from vertex (0-based) to the ID of the
        last edge leaving that vertex.
      * The second one is a map from edge (0-based) to the destination
        node.
      * The third one is a map from edge (0-based) to the next edge ID
        leaving from the same source vertex.
        
    Additionally, to support removing vertices, we keep a set of removed
    vertices. And to support weighted edges, we'll keep a map from edge
    to weight.
    
    Space complexity: O(n + m).
    """
  
    def __init__(self, directed=True):
        """Constructs a graph with 0 nodes and 0 edges."""
        self._vertex_to_last_edge = [] # size n
        self._edge_to_destination = [] # size m
        self._edge_to_next_edge = [] # size m
        self._weights = [] # size m
        self._removed_vertices = set() # see remove_vertex()
        self.n = 0
        self.m = 0
        self.directed = directed

    def display(self):
        """Prints the adjacency lists in a human readable way."""
        print()
        print('{:^8s} | {:^8s}'.format('Vertex', 'Neighbors'))
        print('---------------------------------------------')
        for v in range(self.n):
            if v in self._removed_vertices:
                continue
            print('{:^8d} | {}'.format(v, ', '.join(map(str, sorted(set(self.neighbors(v)))))))
        print()
    
    def debug(self):
        """Prints the adjacency lists."""
        print('Graph:')
        print('  N = %d' % self.n)
        print('  M = %d' % self.m)
        print('  Vertex to last edge: %r' % self._vertex_to_last_edge)
        print('  Edge to destination: %r' % self._edge_to_destination)
        print('  Edge to next edge:   %r' % self._edge_to_next_edge)
        print('  Weights:             %r' % self._weights)
        print('  Removed vertices:    %r' % self._removed_vertices)
  
    def adjacent(self, i, j):
        """Returns true if i and j are directly connected.

        Avg complexity: O(m/n) which is the avg number of
        edge per vertex.
        """
        self._validate(i, j)
        return j in self.neighbors(i)
      
    def neighbors(self, i):
        """Generator for the nodes directly accessible from node i.

        Avg complexity: O(m/n) which is the avg number of
        edge per vertex.
        """
        self._validate(i)
        edge = self._vertex_to_last_edge[i]
        while True:
            if edge == -1:
                return
            n = self._edge_to_destination[edge]
            if n not in self._removed_vertices:
                yield n
            edge = self._edge_to_next_edge[edge]
            
    def add_edge(self, i, j, weight=1.0):
        """Adds an edge from node i to node j.
        
        Allows duplicates!

        Complexity: O(1).
        """
        self._add_edge(i, j, weight, also_other_direction=not self.directed)
  
    def _add_edge(self, i, j, weight, also_other_direction):
        self._validate(i, j)
        self._edge_to_destination.append(j)
        self._weights.append(weight)
        last_edge = self._vertex_to_last_edge[i] # can be -1
        self._edge_to_next_edge.append(last_edge)
        self._vertex_to_last_edge[i] = self.m
        self.m += 1
        if also_other_direction and i != j:
            self._add_edge(j, i, weight, also_other_direction=False)
      
    def remove_edge(self, i, j):
        """Removes edge from node i to node j.
        
        Note, we won't shift the indices, so there will be a "hole".

        Complexity: O(1).
        """
        return self._remove_edge(i, j, also_other_direction=not self.directed)

    def _remove_edge(self, i, j, also_other_direction):
        self._validate(i, j)
        edge = self._vertex_to_last_edge[i]
        last_edge = -1
        while True:
            if edge == -1:
                # i and j are already not connected.
                break
            if self._edge_to_destination[edge] == j:
                # This is the edge to remove.
                if last_edge == -1:
                    # This is also the last edge leaving from i.
                    self._vertex_to_last_edge[i] = self._edge_to_next_edge[edge] # can be -1
                else:
                    self._edge_to_next_edge[last_edge] = self._edge_to_next_edge[edge]
                self._edge_to_next_edge[edge] = -1
                self._edge_to_destination[edge] = -1
                break
            last_edge = edge
            edge = self._edge_to_next_edge[edge]
        if also_other_direction and i != j:
            self._remove_edge(j, i, also_other_direction=False)

    def add_vertex(self):
        """Adds a vertex.

        Complexity: O(1).
        """
        self._vertex_to_last_edge.append(-1)
        self.n += 1

    def remove_vertex(self, i):
        """Removes a vertex.
        
        This is hard, because it would also mean removing edges
        from it (easy), and to it (harder). So we'll just keep
        a list of removed vertices, that will be ignored when
        iterating over neighbors.

        Avg complexity: O(1).
        """
        self._validate(i)
        self._removed_vertices.add(i)

    def _validate(self, *args):
        """Validates index."""
        for i in args:
            if i >= self.n or i < 0:
                raise ValueError('Edges must be between 0 and %d' % self.n)
            if i in self._removed_vertices:
                raise ValueError('Edge %d was removed' % i)

In [48]:
g = Graph()

print('Adding 5 vertices')
g.add_vertex()
g.add_vertex()
g.add_vertex()
g.add_vertex()
g.add_vertex()
g.display()

print('Adding 4->3, 1->1, 1->2, 1->3, 1->4')
g.add_edge(4, 3)
g.add_edge(1, 1)
g.add_edge(1, 2)
g.add_edge(1, 3)
g.add_edge(1, 4)
g.display()

print('Removing 4->3')
g.remove_edge(4, 3)
g.display()

print("0's neighbors: %r" % list(g.neighbors(0)))
print("1's neighbors: %r" % list(g.neighbors(1)))
print('1 connected to 4 ? %r' % g.adjacent(1, 4))
print('4 connected to 1 ? %r' % g.adjacent(4, 1))

print()
print('Removing 1->4')
g.remove_edge(1, 4)
print('Removing vertex 2')
g.remove_vertex(2)
g.display()

print("1's neighbors: %r" % list(g.neighbors(1)))
print('1 connected to 4 ? %r' % g.adjacent(1, 4))

Adding 5 vertices

 Vertex  | Neighbors
---------------------------------------------
   0     | 
   1     | 
   2     | 
   3     | 
   4     | 

Adding 4-3, 1-1, 1-2, 1-3, 1-4

 Vertex  | Neighbors
---------------------------------------------
   0     | 
   1     | 1, 2, 3, 4
   2     | 
   3     | 
   4     | 3

Removing 4-3

 Vertex  | Neighbors
---------------------------------------------
   0     | 
   1     | 1, 2, 3, 4
   2     | 
   3     | 
   4     | 

0's neighbors: []
1's neighbors: [4, 3, 2, 1]
1 connected to 4 ? True
4 connected to 1 ? False

Removing 1-4
Removing vertex 2

 Vertex  | Neighbors
---------------------------------------------
   0     | 
   1     | 1, 3
   3     | 
   4     | 

1's neighbors: [3, 1]
1 connected to 4 ? False


In [51]:
g = Graph(directed=False)

print('Adding 5 vertices')
g.add_vertex()
g.add_vertex()
g.add_vertex()
g.add_vertex()
g.add_vertex()
g.display()

print('Adding 4-3, 1-1, 1-2, 1-3, 1-4')
g.add_edge(4, 3)
g.add_edge(1, 1)
g.add_edge(1, 2)
g.add_edge(1, 3)
g.add_edge(1, 4)
g.display()

print('Removing 4-3')
g.remove_edge(4, 3)
g.display()

print("0's neighbors: %r" % list(g.neighbors(0)))
print("1's neighbors: %r" % list(g.neighbors(1)))
print('1 connected to 4 ? %r' % g.adjacent(1, 4))
print('4 connected to 1 ? %r' % g.adjacent(4, 1))

print()
print('Removing 1-4')
g.remove_edge(1, 4)
print('Removing vertex 2')
g.remove_vertex(2)
g.display()

print("1's neighbors: %r" % list(g.neighbors(1)))
print('1 connected to 4 ? %r' % g.adjacent(1, 4))

Adding 5 vertices

 Vertex  | Neighbors
---------------------------------------------
   0     | 
   1     | 
   2     | 
   3     | 
   4     | 

Adding 4-3, 1-1, 1-2, 1-3, 1-4

 Vertex  | Neighbors
---------------------------------------------
   0     | 
   1     | 1, 2, 3, 4
   2     | 1
   3     | 1, 4
   4     | 1, 3

Removing 4-3

 Vertex  | Neighbors
---------------------------------------------
   0     | 
   1     | 1, 2, 3, 4
   2     | 1
   3     | 1
   4     | 1

0's neighbors: []
1's neighbors: [4, 3, 2, 1]
1 connected to 4 ? True
4 connected to 1 ? True

Removing 1-4
Removing vertex 2

 Vertex  | Neighbors
---------------------------------------------
   0     | 
   1     | 1, 3
   3     | 1
   4     | 

1's neighbors: [3, 1]
1 connected to 4 ? False


In [58]:
def dfs(g, func, verbose=False):
    """Apply depth-first search to graph g pre-order.

    g is assumed to be connected.
    """
    stack = [0]
    visited = set([0])
    while stack:
        if verbose:
            print('Stack: %r' % stack)
        v = stack.pop()
        func(v)
        neighbors = sorted(n for n in g.neighbors(v) if n not in visited)
        if verbose:
            print('Adding: %r' % neighbors)
        stack.extend(neighbors)
        visited |= set(neighbors)

In [59]:
g = Graph()
for _ in range(11):
    g.add_vertex()
g.add_edge(0, 2)
g.add_edge(0, 4)
g.add_edge(4, 6)
g.add_edge(4, 8)
g.add_edge(8, 10)
g.add_edge(2, 1)
g.add_edge(4, 3)
g.add_edge(6, 5)
g.add_edge(8, 7)
g.add_edge(10, 9)
g.add_edge(0, 1)
g.add_edge(2, 3)
g.add_edge(4, 5)
g.add_edge(6, 7)
g.add_edge(8, 9)
g.add_edge(10, 0)
g.display()

dfs(g, lambda v: print('Visiting %d' % v), verbose=True)
print()
dfs(g, lambda v: print('Visiting %d' % v))


 Vertex  | Neighbors
---------------------------------------------
   0     | 1, 2, 4
   1     | 
   2     | 1, 3
   3     | 
   4     | 3, 5, 6, 8
   5     | 
   6     | 5, 7
   7     | 
   8     | 7, 9, 10
   9     | 
   10    | 0, 9

Stack: [0]
Visiting 0
Adding: [1, 2, 4]
Stack: [1, 2, 4]
Visiting 4
Adding: [3, 5, 6, 8]
Stack: [1, 2, 3, 5, 6, 8]
Visiting 8
Adding: [7, 9, 10]
Stack: [1, 2, 3, 5, 6, 7, 9, 10]
Visiting 10
Adding: []
Stack: [1, 2, 3, 5, 6, 7, 9]
Visiting 9
Adding: []
Stack: [1, 2, 3, 5, 6, 7]
Visiting 7
Adding: []
Stack: [1, 2, 3, 5, 6]
Visiting 6
Adding: []
Stack: [1, 2, 3, 5]
Visiting 5
Adding: []
Stack: [1, 2, 3]
Visiting 3
Adding: []
Stack: [1, 2]
Visiting 2
Adding: []
Stack: [1]
Visiting 1
Adding: []

Visiting 0
Visiting 4
Visiting 8
Visiting 10
Visiting 9
Visiting 7
Visiting 6
Visiting 5
Visiting 3
Visiting 2
Visiting 1


In [55]:
class Queue:
    """Quick and dirty fixed-size queue, used for BFS."""

    def __init__(self, size):
        self.n = size + 1
        self.front = 0
        self.back = 0
        self.array = [0] * (size + 1)

    def __repr__(self):
        return '%s' % self

    def __str__(self):
        if self.front <= self.back:
            values = self.array[self.front + 1 : self.back + 1]
        else:
            values = self.array[self.front + 1 :] + self.array[: self.back + 1]
        return str(values)

    def enqueue(self, x):
        self.back = (self.back + 1) % self.n
        assert self.back != self.front, 'queue full'
        self.array[self.back] = x

    def dequeue(self):
        assert self.back != self.front, 'queue empty'
        self.front = (self.front + 1) % self.n
        return self.array[self.front]

    def empty(self):
        return self.back == self.front

In [60]:
def bfs(g, func, verbose=False):
    """Apply breadth-first search to graph g pre-order.

    g is assumed to be connected.
    """
    queue = Queue(g.n)
    queue.enqueue(0)
    visited = set([0])
    while not queue.empty():
        if verbose:
            print('Queue: %r' % queue)
        v = queue.dequeue()
        func(v)
        neighbors = sorted(n for n in g.neighbors(v) if n not in visited)
        if verbose:
            print('Adding: %r' % neighbors)
        for n in neighbors:
            queue.enqueue(n)
            visited |= set(neighbors)

In [61]:
g = Graph()
for _ in range(11):
    g.add_vertex()
g.add_edge(0, 2)
g.add_edge(0, 4)
g.add_edge(4, 6)
g.add_edge(4, 8)
g.add_edge(8, 10)
g.add_edge(2, 1)
g.add_edge(4, 3)
g.add_edge(6, 5)
g.add_edge(8, 7)
g.add_edge(10, 9)
g.add_edge(0, 1)
g.add_edge(2, 3)
g.add_edge(4, 5)
g.add_edge(6, 7)
g.add_edge(8, 9)
g.add_edge(10, 0)
g.display()

bfs(g, lambda v: print('Visiting %d' % v), verbose=True)
print()
bfs(g, lambda v: print('Visiting %d' % v))


 Vertex  | Neighbors
---------------------------------------------
   0     | 1, 2, 4
   1     | 
   2     | 1, 3
   3     | 
   4     | 3, 5, 6, 8
   5     | 
   6     | 5, 7
   7     | 
   8     | 7, 9, 10
   9     | 
   10    | 0, 9

Queue: [0]
Visiting 0
Adding: [1, 2, 4]
Queue: [1, 2, 4]
Visiting 1
Adding: []
Queue: [2, 4]
Visiting 2
Adding: [3]
Queue: [4, 3]
Visiting 4
Adding: [5, 6, 8]
Queue: [3, 5, 6, 8]
Visiting 3
Adding: []
Queue: [5, 6, 8]
Visiting 5
Adding: []
Queue: [6, 8]
Visiting 6
Adding: [7]
Queue: [8, 7]
Visiting 8
Adding: [9, 10]
Queue: [7, 9, 10]
Visiting 7
Adding: []
Queue: [9, 10]
Visiting 9
Adding: []
Queue: [10]
Visiting 10
Adding: []

Visiting 0
Visiting 1
Visiting 2
Visiting 4
Visiting 3
Visiting 5
Visiting 6
Visiting 8
Visiting 7
Visiting 9
Visiting 10
